## 6. Angriest-Krueger (1991) Replication

You'll find data from a famous paper by Angrist and Krueger (1991) in the ARE212_Materials repository, along with a pdf of the paper. The paper uses information on quarter of birth as an instrument for (endogenous) education to measure returns to education. The first specification in the paper is given in their equations (1) and (2).

#### 1. What is the (implicit) identifying assumption? Comment on its plausibility.

#### 2. Using their data, estimate (2), replicating the figures in their Table 5, using the conventional two-stage least squares IV estimator (what they call TSLS).

In [73]:
import pandas as pd
%matplotlib inline
import numpy as np
from scipy.stats import multivariate_normal
from scipy.linalg import inv, sqrtm
import numpy as np

# read in dta file
file_path = '~/ARE212_Materials/angrist-krueger91.dta'
df = pd.read_stata(file_path)

# quick look at data
print(df.head)

<bound method NDFrame.head of          ageq  edu   logwage  married  state  qob  black  smsa   yob  region
0       47.00   12  6.245846        1      1    1      1     1  1933     0.0
1       46.25   12  5.847161        1     48    4      1     1  1933     0.0
2       50.00   12  6.645516        1      2    1      1     1  1930     0.0
3       47.00   16  6.706133        1     22    1      1     1  1933     0.0
4       42.25   14  6.357876        1     42    4      1     1  1937     0.0
...       ...  ...       ...      ...    ...  ...    ...   ...   ...     ...
329504  42.50   10  4.583833        1     26    3      1     1  1937     4.0
329505  42.00   12  5.784210        1     22    1      1     1  1938     4.0
329506  41.00   12  5.707302        1     48    1      1     1  1939     4.0
329507  47.25   12  5.952494        1     42    4      1     1  1932     4.0
329508  48.50   13  6.047781        1     20    3      1     1  1931     4.0

[329509 rows x 10 columns]>


In [86]:
#create instrumental variable
qob_yob_int = df.qob*df.yob
df = pd.concat([df, qob_yob_int], axis=1)

# set up dummy variables
dummy_var_qobyob = pd.get_dummies(df['qob_yob_int'], prefix='qob_yob_int')
dummy_var_yob = pd.get_dummies(df['yob'], prefix='yob')
dummy_var_region = pd.get_dummies(df['region'], prefix='region')

# concatenate the original DataFrame with the dummy variables
df = pd.concat([df, dummy_var_qobyob, dummy_var_yob, dummy_var_region], axis=1)

# create new data frame with only men 30-39.
df_tab5 = df[(df['ageq'] >= 30) & (df['ageq'] <= 39)]
df_tab5 = df_tab5.reset_index(drop=True)

# define variables
df['intercept'] = 1
y = df.logwage
X = df[['intercept', 'edu', 'dummy_var_yob']]
instrument = df['qob_yob_int']

KeyError: "['dummy_var_yob'] not in index"

In [55]:
# regression
b = np.linalg.solve(X.T@X,X.T@y)

e = y - X@b

vb = e.var()*inv(X.T@X)

print(b)
print(np.sqrt(np.diag(vb)))


[-5.24445623e-13 -7.26036770e-15  1.00000000e+00]
[1.02294476e-15 3.76948760e-17 1.82206443e-16]


In [64]:
#### vague attempt at setting up reg
model = sm.OLS(y, X).fit()

# Print the regression summary
results = model.summary(alpha=0.05)
print(results)

                            OLS Regression Results                            
Dep. Variable:                logwage   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 8.256e+33
Date:                Sat, 30 Mar 2024   Prob (F-statistic):               0.00
Time:                        17:12:05   Log-Likelihood:             1.0548e+07
No. Observations:              329509   AIC:                        -2.110e+07
Df Residuals:                  329506   BIC:                        -2.110e+07
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
intercept  -1.674e-14   4.65e-17   -359.872      0.0